# 1. Introduction


In this work, I'm going to predict the house pricing in U.S. I'm going to provide multiple solutions for this project by trying different algorithms in order to get better result. This is supervised approch, we will start from Linear regression and go to more advaced ways for instance Random Forest. This work depnds on House Sales in King County, USA. This dataset is provided by kaggle.com | https://www.kaggle.com/harlfoxem/housesalesprediction/data


# 2. Dependencies¶


In [1]:

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import ExtraTreesRegressor,GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import validation_curve
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score


In [22]:
DOWNLOAD_ROOT = "../input/kc_house_data.csv"

def create_dataframe(data_path):
    df = pd.read_csv(data_path)
    return df

In [23]:
housing = create_dataframe(DOWNLOAD_ROOT)
housing.head()

# 3. Data exploratory¶


In [24]:
housing.info()


as we can see the data contains 21 columns such as date, price, floors ...etc.



lets see if we have any missing value or null



In [25]:
print(housing.isnull().any())


It seems the data is almost clean no missing values



In [26]:
with sns.plotting_context("notebook",font_scale=2.5):
    g = sns.pairplot(housing[['sqft_lot','sqft_above','price','sqft_living','bedrooms']], 
                 hue='bedrooms', palette='tab20',size=6)
g.set(xticklabels=[])

Seaborn is a Python visualization library based on matplotlib. It provides a high-level interface for drawing attractive statistical graphics(official). As we see we got some linear visulas, so I will try to use some linear model and see what I can get

In [27]:
%matplotlib inline
import matplotlib.pyplot as plt
housing.hist(bins=10, figsize=(20,15))
plt.show()

In [28]:
import seaborn as sns
corr = housing.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)

As we know the correlation coefficient ranges from –1 to 1, so here we can observe that there is strong postive correlation between price and soqft_lot , between price and grade and between batheroom and soft_living. However, we can see that strong negative correlation between floors and condition. When coefficients close to zero mean that there is no linear correlation such as condition and zipcode.

In [29]:
housing.plot(kind="scatter", x="long", y="lat",alpha=0.1)


# 6. Prepare the data

Now I'm going to split the data into train_set and test_set. I'm going to give the ratio 0.2, meaning that 80% for train_set and 20% for test_set, I will keep the test_set aside in order to test our model, but before that , lets try to scale the data as it is proofed that some algorithms perform well on scaled data. In addition, I will keep copy of data non scaled in order to try it with algorithms which don't perform well on scaled data


In [32]:
#before scaling the data I'm going to deop price col since it is a label then droping date and id since does't make any
housing_scaled = housing.drop('price', axis=1)  # drop labels for training set
housing_scaled = housing_scaled.drop('date', axis=1)  # drop date  for training set
housing_scaled = housing_scaled.drop('id', axis=1)  # drop id  for training set

#extract the labels
housing_labels = housing['price'].copy()
#scale the data with the label
scaler = preprocessing.StandardScaler().fit(housing_scaled)
new_df = scaler.transform(housing_scaled)
housing_scaled = pd.DataFrame(data=new_df, index=list(range(len(new_df))), columns=housing_scaled.columns)

In [33]:
housing_scaled.head()



In [35]:
np.random.seed(0) 
#split the data with raito 0.2
X_train, X_test, y_train, y_test = train_test_split(housing_scaled.values, housing_labels.values, test_size=0.2)


In [37]:
print (X_train.shape)
print (y_train.shape)
print (X_test.shape)
print (y_test.shape)

# 7.Select the model and Evaluation¶

# 7.1 linear regression with Ridge regularizer:¶

In [38]:
from sklearn.linear_model import Ridge
ridge_reg=Ridge(alpha=1,solver="cholesky")
ridge_reg.fit(X_train,y_train)



In [39]:
ridge_reg.score(X_test, y_test)



In [40]:
ridge_reg.score(X_train, y_train)



In [41]:
y_pred=ridge_reg.predict(X_test)
r2_score(y_test, y_pred)



# 7.2 KNeighbors¶

In [42]:
from sklearn.neighbors import KNeighborsRegressor
knn_rg = KNeighborsRegressor(n_neighbors=3)
knn_rg.fit(X_train, y_train)

In [43]:
knn_rg.score(X_test, y_test)



In [44]:
knn_rg.score(X_train, y_train)

In [45]:
y_pred=knn_rg.predict(X_test)
r2_score(y_test, y_pred)



# 7.3 Trainig with SGD¶

In [46]:
from sklearn.linear_model import SGDRegressor
#trying SGD with penalty l2
sgd_reg=SGDRegressor(max_iter=500,penalty='l2',eta0=0.1)
sgd_reg.fit(X_train,y_train.ravel())



In [47]:
sgd_reg.score(X_test, y_test)



In [48]:
sgd_reg.score(X_train, y_train)



In [49]:
y_pred=sgd_reg.predict(X_test)
r2_score(y_test, y_pred)



# 7.4 RandomForestRegressor:¶

In [50]:
from sklearn.ensemble import RandomForestRegressor
forest_reg_clf = RandomForestRegressor(random_state=0)
forest_reg_clf.fit(X_train,y_train)



In [51]:
forest_reg_clf.score(X_test, y_test)



In [52]:
forest_reg_clf.score(X_train, y_train)



In [53]:
y_pred=forest_reg_clf.predict(X_test)
r2_score(y_test, y_pred)



# 7.5 RandomForestRegressor with GridSearchCV¶

In [54]:
param_grid=[
{'n_estimators':[3,10,30,40,50],	'max_features':[10,14,18]},
{'n_estimators':[3,10],'max_features':[14,18]}]
forest_reg=RandomForestRegressor(random_state=0,n_jobs=-1)
rnd_grid_search=GridSearchCV(forest_reg,param_grid,cv=10)
rnd_grid_search.fit(X_train,y_train)



In [55]:
rnd_grid_search.cv_results_['mean_test_score']



In [56]:
rnd_grid_search.cv_results_['mean_train_score']



In [57]:
rnd_grid_search.score(X_test, y_test)



In [58]:
rnd_grid_search.score(X_train, y_train)



In [59]:
y_pred=rnd_grid_search.predict(X_test)
r2_score(y_test, y_pred)



# 7.6 GradientBoostingRegressor:¶

trains a GBRT ensemble with 120 trees, then measures the validation error at each stage of training to find the optimal number of trees, and finally trains another GBRT ensemble using the optimal number of trees


In [60]:
gbrt=GradientBoostingRegressor(max_depth=3,	n_estimators=120)
gbrt.fit(X_train,y_train)
errors=[mean_squared_error(y_test,y_pred)
for y_pred in gbrt.staged_predict(X_test)]
bst_n_estimators=np.argmin(errors)
param_grid=[
{'n_estimators':[bst_n_estimators],'max_features':[2,4,6,8],'max_depth':[1,2,3],'learning_rate':[0.1,0.2,0.5],'random_state':[0]}]

gbrt_best=GradientBoostingRegressor()
gbrt_grid_search=GridSearchCV(gbrt_best,param_grid,cv=5)

gbrt_grid_search.fit(X_train,y_train)



In [61]:
gbrt_grid_search.cv_results_['mean_test_score']



In [62]:
gbrt_grid_search.cv_results_['mean_train_score']



In [63]:
gbrt_grid_search.score(X_test, y_test)



In [64]:
gbrt_grid_search.score(X_train, y_train)



In [65]:
y_pred=gbrt_grid_search.predict(X_test)
r2_score(y_test, y_pred)



# 7.7 Extra Trees Regressor¶

Extra tree trades more bias for a lower variance. It also makes Extra-Trees much faster to train than regular Random Forests since finding the best possible threshold for each feature at every node is one of the most time-consuming tasks of growing a tree


In [66]:
param_grid=[
{'n_estimators':[3,10,30,40,50],'max_features':[2,4,6,8]},
{'n_estimators':[3,10],'max_features':[2,3,4]}]
forest_reg_extra=ExtraTreesRegressor()
rnd_grid_search_extra=GridSearchCV(forest_reg_extra,param_grid,cv=10)
rnd_grid_search_extra.fit(X_train,y_train)



In [67]:
rnd_grid_search_extra.cv_results_['mean_test_score']



In [68]:
rnd_grid_search_extra.cv_results_['mean_train_score']



In [69]:
rnd_grid_search_extra.score(X_test, y_test)



In [70]:
rnd_grid_search_extra.score(X_train, y_train)



In [71]:
y_pred=rnd_grid_search_extra.predict(X_test)
r2_score(y_test, y_pred)



# 8.Discussion¶

# 8.1 linear regression with Ridge regularizer:¶

First of all, the first model trained is linear regression with ridge regularization, but as we have seen we got very bad accuracy about 69% lets try to pot the actual value and predicted value to see what is happing 

In [72]:
# Visualising the results
housing_predictions_ridge_reg=ridge_reg.predict(X_test)
plt.plot(y_test, color = 'red', label = 'house Price')
plt.plot(housing_predictions_ridge_reg, color = 'blue', label = 'Predicted house Price')
plt.title(' house Price Prediction')

plt.legend()
plt.show()



ok as we can see the model is not predicting very well because the model is too simple,lets try to plot residual(The difference between the observed value of the dependent variable (y) and the predicted value (ŷ) is called the residual)


In [73]:
housing_predictions=ridge_reg.predict(X_test)



In [74]:
# housing_predictions=ridge_reg.predict(X_test)
housing_predictions_train=ridge_reg.predict(X_train)
plt.scatter(housing_predictions_train, housing_predictions_train - y_train, c = "blue", marker = "s", label = "Training data")
plt.scatter(housing_predictions, housing_predictions - y_test, c = "lightgreen", marker = "s", label = "Validation data")
plt.title("Linear regression")
plt.xlabel("Predicted values")
plt.ylabel("Residuals")
plt.legend(loc = "upper left")
plt.hlines(y = 0, xmin = 11.5, xmax = 15.5, color = "red")
plt.show()



The residual plot shows a fairly random pattern,This random pattern indicates that a linear model provides a decent fit to the data. As we see here not extacly random that means our data not pure linear


# 8.2 KNeighbors¶

The second model is KNN,here we got accuracy about 78%


In [75]:
# Visualising the results
housing_predictions_knn_rg=knn_rg.predict(X_test)
plt.plot(y_test, color = 'red', label = 'house Price')
plt.plot(housing_predictions_knn_rg, color = 'blue', label = 'Predicted house Price')
plt.title(' house Price Prediction')

plt.legend()
plt.show()



Well it is better than linear regression, but still this model not predicting very well. For example the prices which as above 5 milion but KNN predict them as 2.5 million


# 8.3 GradientBoostingRegressor

As mentioned, I trained a GBRT ensemble with 120 trees, then measures the validation error at each stage of training to find the optimal number of trees, and finally trains another GBRT ensemble using the optimal number of trees


In [76]:
gbrt_grid_search.best_estimator_



# 8.4 RandomForestRegressor¶

Finally, It seems that Random Forest Model predict very well without overfitting. Because this model consist of ensemble meaning that it contains many decision trees which make it very powerfull.Lets try to plot the actual values and predicted values


In [77]:
# Visualising the results
housing_predictions_forest_reg_clf=forest_reg_clf.predict(X_test)
plt.plot(y_test, color = 'red', label = 'house Price')
plt.plot(housing_predictions_forest_reg_clf, color = 'blue', label = 'Predicted house Price')
plt.title(' house Price Prediction')

plt.legend()
plt.show()



In [78]:
#plotting a scatter plot between the real price  values and predict price  values

plt.scatter(housing_predictions_forest_reg_clf, y_test, c='red', alpha=0.5)
plt.show()



For better evaluation and in order to avoid any overfitting, I have tried Random Forest with Cross Validation and Grid Search


In [79]:
# Visualising the results
housing_predictions_rnd_grid_search=rnd_grid_search.predict(X_test)
plt.plot(y_test, color = 'red', label = 'house Price')
plt.plot(housing_predictions_rnd_grid_search, color = 'blue', label = 'Predicted house Price')
plt.title(' house Price Prediction')

plt.legend()
plt.show()



In [80]:
#plotting a scatter plot between the real price  values and predict price  values

plt.scatter(housing_predictions_rnd_grid_search, y_test, c='red', alpha=0.5)
plt.show()



Ok, it seems the pervious model is overfitting, here we can see the best hyperparametes


In [81]:
rnd_grid_search.best_estimator_



# 9. Conclusion






In this work, I tried multiple algorithms such as linear regression with Ridge regularizer and I got performance around 69% which is poor result. Moreover, I tried KNN algorithm, the performance increased to 77%. However. In RandomForestRegressor with best hyperparameters and cross validation, I achieved good improvment when I got accuracy about 89%.
